# Итетарторы и генераторы

### Как работает цикл for?

In [15]:
for index in range(10):
    pass

for k, v in {1: 1, 2: 2}.items():
    pass

try:
    for line in open("test.py"):
        pass
except Exception:
    pass

for sym in "some letters":
    pass

In [7]:
describe = lambda x: set(dir(x))

In [9]:
describe([]) & describe({}) & describe("") & describe(set()) - describe(5)

{'__contains__', '__iter__', '__len__'}

\__contains__ - работает, когда вызывается конструкция <i>el in b</i>

\__len__ - длина элемента

\__iter__ -?

### Функции iter и next

In [35]:
els = list(range(5))
it_els = iter(els)
# it_els = els.__iter__()

In [36]:
type(els), type(it_els), type(iter(it_els))

(list, list_iterator, list_iterator)

In [37]:
next(it_els), it_els.__next__(), next(it_els), next(it_els)

(0, 1, 2, 3)

In [38]:
print(*iter(it_els))

4


In [39]:
next(it_els)

StopIteration: 

### Собственные итерируемые объекты

In [27]:
class RangeIterator(object):
    def __init__(self, i, j):
        self.i = i #первое число
        self.j = j #последнее число
 
    def __iter__(self):
        return self
 
    def __next__(self):
        if self.i < self.j:
            ret_val = self.i
            self.i += 1
            return ret_val
        else:
            raise StopIteration("No more elements")
 
    pass

it = RangeIterator(10, 20)
print([x for x in it])

10 11 12 13 14 15 16 17 18 19


### Последовательности

In [29]:
class Indexer(object):
    def __init__(self, max_index):
        self.max_index = max_index
    
    
    def __getitem__(self, idx):
        if idx > self.max_index:
            raise IndexError(idx)
        return idx
    pass

In [30]:
it = Indexer(5)
print([x for x in it])

[0, 1, 2, 3, 4, 5]


### Исчерпаемость

In [52]:
class RangeIterator1(object):
    def __init__(self, i, j):
        self.i = i #первое число
        self.j = j #последнее число
    def __iter__(self):
        return self
    def __next__(self):
        if self.i < self.j:
            ret_val = self.i
            self.i += 1
            return ret_val
        else:
            raise StopIteration("No more elements")
    pass
it = RangeIterator(10, 20)
print(list(it), list(it))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19] []


In [54]:
class RangeIteratorHelper(object):
    def __init__(self, i, j):
        self.i = i #первое число
        self.j = j #последнее число
    def __next__(self):
        if self.i < self.j:
            ret_val = self.i
            self.i += 1
            return ret_val
        else:
            raise StopIteration("No more elements")
    pass

class RangeIterator2(object):
    def __init__(self, i, j):
        self.i = i #первое число
        self.j = j #последнее число
    def __iter__(self):
        return RangeIteratorHelper(self.i, self.j)
    pass
it = RangeIterator2(10, 20)
print(list(it), list(it), sep="\n")

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


### Устройство цикла for

In [57]:
els = list(range(5))
it_els = iter(els)

In [58]:
def some_action(value):
    print(value)

while True:
    try:
        value = next(it_els)
    except StopIteration:
        break
    some_action(value)

0
1
2
3
4


#### что то же

In [59]:
els = list(range(5))
it_els = iter(els)

In [60]:
for value in it_els:
    some_action(value)

0
1
2
3
4


### x in Iterable

In [61]:
class Dict(dict):
    def __contains__(self, target):
        for item in self:
            if item == target:
                return True
        return False

In [65]:
d = Dict({1:1, 2:3})
1 in d

True

### Пример генератора

In [11]:
def some_gen(x):
    yield x
    x += 10
    yield x
    print("End of generator")
    
gen = some_gen(10)
print(type(gen))
print(next(gen)), print(list(gen)), print(next(gen)), print(list(gen))

<class 'generator'>
10
End of generator
[20]


StopIteration: 

In [3]:
def fibonacci(max_num=10):
    a, b = 0, 1
    while a < max_num:
        # return a, + запоминает место рестарта для следующего вызова
        yield a
        a, b = b, a + b
    pass

# Используем генератор как итератор
[number for number in fibonacci()]

[0, 1, 1, 2, 3, 5, 8]

In [13]:
def even(iterable):
    for idx in iterable:
        if not idx % 2:
            yield idx

for idx in even(range(10)):
    print(idx)

0
2
4
6
8


### map, zip, filter

#### Напишите функцию map

In [38]:
def _map(function, iterable):
    for item in iterable:
        yield function(item)
    pass

In [40]:
_ = list(_map(print, [1, 2, 3]))

1
2
3


In [47]:
_ = list(map(print, [1, 2, 3]))

1
2
3


#### Напишите функцию filter

In [48]:
def _filter(function, iterable):
    for item in iterable:
        if function(item):
            yield item
    pass

In [49]:
_ = list(_map(print, _filter(lambda x: x % 2, range(10))))

1
3
5
7
9


In [50]:
_ = list(_map(print, filter(lambda x: x % 2, range(10))))

1
3
5
7
9


#### Напишите функцию zip

In [51]:
def _zip(*iterables):
    def zipped(iterables):
        while True:
            res = []
            end = False
            for iterable in iterables:
                try:
                    res.append(next(iterable))
                except StopIteration:
                    end = True
                    break
            if end:
                break
            else:
                yield tuple(res)
    return zipped(list(_map(iter, iterables)))

In [52]:
_ = list(map(print, _zip([1, 2, 3], range(4))))

(1, 0)
(2, 1)
(3, 2)


In [53]:
_ = list(map(print, zip([1, 2, 3], range(4))))

(1, 0)
(2, 1)
(3, 2)


### Задача

Дана последовательность N чисел. Необходимо посчитать суммы всех идущих друг за другом под-последовательностей длины M. Т. е. если на вход подается range(9), то ответом будет [3, 12, 21].

#### Решение

In [ ]:
<your code here>

### Цепочки

In [60]:
def chain(*iterables):
    for iterable in iterables:
        for it in iterable:
            yield it
list(chain(range(5), [10, 20], "test"))

[0, 1, 2, 3, 4, 10, 20, 't', 'e', 's', 't']

### Функция enumerate

In [72]:
def _enumerate(iterable):
    i = 0
    for it in iterable:
        yield i, it
        i += 1
        
list(_enumerate("test"))

[(0, 't'), (1, 'e'), (2, 's'), (3, 't')]

### Применение генераторов

In [ ]:
class BinaryTree:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left, self.right = left, right

    def __iter__(self):
        for node in self.left:
            yield node.value
        yield self.value
        for node in self.right:
            yield node.value

### yield from 

In [ ]:
class BinaryTree:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left, self.right = left, right

    def __iter__(self):
        if self.left: yield from self.left
        yield self.value
        if self.right: yield from self.right

In [74]:
def g(x):
    yield from range(x, 0, -1)
    yield from range(x)

list(g(5))

[5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

### send и throw

In [107]:
def echo(value=None):
    print("Execution starts when 'next()' is called for the first time.")
    try:
        while True:
            try:
                value = (yield value)
            except Exception as e:
                value = e
    finally:
        print("Don't forget to clean up when 'close()' is called.")

generator = echo(1)
print(next(generator))

print(next(generator))

print(generator.send(2))

generator.throw(TypeError, "spam")

generator.close()

Execution starts when 'next()' is called for the first time.
1
None
2
Don't forget to clean up when 'close()' is called.


In [93]:
next(generator)

StopIteration: 

### Еще пример генератора

In [89]:
def accumulate():
    tally = 0
    while 1:
        next = yield
        if next is None:
            return tally
        tally += next

def gather_tallies(tallies):
    while 1:
        tally = yield from accumulate()
        tallies.append(tally)

tallies = []
acc = gather_tallies(tallies)
# Ensure the accumulator is ready to accept values
next(acc)
for i in range(4):
    acc.send(i)

acc.send(None)  # Finish the first tally
for i in range(5):
    acc.send(i)

acc.send(None)  # Finish the second tally
acc.close()
tallies

[6, 10]

### Выражения генераторы

In [114]:
%%time
a = filter(lambda x : not (x % 33), (x * (x + 2) for x in range(10 ** 7)))

Wall time: 177 ms


In [115]:
%%time
a = filter(lambda x : not (x % 33), [x * (x + 2) for x in range(10 ** 7)])

Wall time: 1.84 s


### Itertools

https://docs.python.org/3/library/itertools.html